In [1]:
# НАЙТИ АУДИО ИЗ ДАТАСЕТА С WER 0
# ПОСТРОИТЬ GENERATE С ЭТИМ АУДИО БЕЗ BATCH DECODE - НУЖНЫ ТОКЕНЫ
# ВЗЯТЬ AUDIO + ТЕКСТ И С ПОМОЩЬЮ DATASET ЗАГРУЗИТЬ ИХ - ПРОВЕРИТЬ ЧТО ТОКЕНЫ СОВПАДАЮТ
# ПРОБЛЕМА ДОЛЖНА БЫТЬ В НАЧАЛЬНЫХ ТОКЕНАХ

In [4]:
import sys
sys.path.append('..')

import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from src.data import DataManager
from src.config import load_config
from jiwer import wer
from tqdm import tqdm

In [ ]:
# БЛОК 1: Загрузка модели и поиск sample с WER=0



print("Загрузка модели и процессора...")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

print(f"Device: {device}")

# Загрузка конфига и датасета
print("\nЗагрузка датасета...")
config = load_config("../configs/whisper_base.yaml")
data_manager = DataManager(config)
data_manager.set_already_loaded_processor(processor)

# Создаем train датасет
train_dataset = data_manager.create_dataset('train')  # Ограничим для скорости
print(f"Датасет загружен: {len(train_dataset)} samples")

# Ищем sample с WER=0
print("\nПоиск sample с WER=0...")
perfect_sample_idx = None

for idx in tqdm(range(len(train_dataset))):
    try:
        # Получаем sample из датасета
        sample = train_dataset[idx]
        
        # Получаем reference текст (ключ "text", а не "reference")
        reference = sample['text']
        
        # Инференс модели
        input_features = sample['input_features'].unsqueeze(0).to(device)
        
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                language="ru",
                task="transcribe",
                max_length=448
            )
        
        # Декодируем без skip_special_tokens для текстового сравнения
        prediction = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # Вычисляем WER
        sample_wer = wer(reference, prediction)
        
        if sample_wer == 0.0:
            perfect_sample_idx = idx
            print(f"\n✅ Найден sample с WER=0 на индексе {idx}")
            print(f"Reference: {reference}")
            print(f"Prediction: {prediction}")
            break
            
    except Exception as e:
        print(f"Ошибка на sample {idx}: {e}")
        continue

if perfect_sample_idx is None:
    print("\n⚠️ Sample с WER=0 не найден в первых 500 samples")
    print("Попробуйте увеличить limit или использовать validation датасет")
else:
    print(f"\n🎯 Используем sample #{perfect_sample_idx} для дальнейшего анализа")

Загрузка модели и процессора...
Device: cuda

Загрузка датасета...
Датасет загружен: 26654 samples

Поиск sample с WER=0...


  0%|          | 31/26654 [00:04<1:10:01,  6.34it/s]


✅ Найден sample с WER=0 на индексе 31
Reference: Спрятаться тут негде.
Prediction:  Спрятаться тут негде.

🎯 Используем sample #31 для дальнейшего анализа


In [3]:
# БЛОК 2: Generate для найденного sample - вывод ТОКЕНОВ (с спец токенами)

if perfect_sample_idx is None:
    print("⚠️ Пропускаем - sample не найден")
else:
    print(f"Анализ sample #{perfect_sample_idx}")
    print("="*80)
    
    # Получаем sample
    sample = train_dataset[perfect_sample_idx]
    input_features = sample['input_features'].unsqueeze(0).to(device)
    
    # Generate - получаем токены
    print("\n🔮 GENERATE (модель):")
    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            language="ru",
            task="transcribe",
            max_length=448
        )
    
    # Выводим токены
    predicted_ids_list = predicted_ids[0].cpu().tolist()
    print(f"\nКоличество токенов: {len(predicted_ids_list)}")
    print(f"\nПервые 20 токенов: {predicted_ids_list[:20]}")
    print(f"\nПоследние 20 токенов: {predicted_ids_list[-20:]}")
    
    # Декодируем БЕЗ skip_special_tokens
    predicted_text_with_special = processor.decode(predicted_ids[0], skip_special_tokens=False)
    print(f"\nТекст с спец токенами:\n{repr(predicted_text_with_special)}")
    
    # Декодируем С skip_special_tokens для сравнения
    predicted_text = processor.decode(predicted_ids[0], skip_special_tokens=True)
    print(f"\nТекст БЕЗ спец токенов:\n{predicted_text}")
    
    print("\n" + "="*80)

Анализ sample #31

🔮 GENERATE (модель):

Количество токенов: 13

Первые 20 токенов: [2933, 14566, 2229, 1008, 776, 5243, 681, 11, 1725, 2344, 678, 14016, 13]

Последние 20 токенов: [2933, 14566, 2229, 1008, 776, 5243, 681, 11, 1725, 2344, 678, 14016, 13]

Текст с спец токенами:
' Спереден сентя, не льди.'

Текст БЕЗ спец токенов:
 Спереден сентя, не льди.



In [4]:
# БЛОК 3: Загрузка того же sample через dataset - вывод labels (с спец токенами)

if perfect_sample_idx is None:
    print("⚠️ Пропускаем - sample не найден")
else:
    print(f"\nАнализ labels из датасета для sample #{perfect_sample_idx}")
    print("="*80)
    
    # Получаем sample из датасета
    sample = train_dataset[perfect_sample_idx]
    
    # Получаем labels (токенизированные внутри датасета)
    labels = sample['labels']  # Tensor shape: (sequence_length,)
    
    print("\n📋 LABELS (из датасета):")
    
    # Преобразуем в список, заменяя -100 на pad_token_id для декодирования
    labels_list = labels.cpu().tolist()
    
    # Фильтруем -100 (padding используемый для loss)
    labels_without_ignore = [token_id if token_id != -100 else processor.tokenizer.pad_token_id 
                              for token_id in labels_list]
    
    print(f"\nКоличество токенов (включая -100): {len(labels_list)}")
    print(f"Количество токенов без -100: {len([x for x in labels_list if x != -100])}")
    
    print(f"\nПервые 20 токенов: {labels_list[:20]}")
    print(f"\nПоследние 20 токенов: {labels_list[-20:]}")
    
    # Декодируем БЕЗ skip_special_tokens
    # Используем только не -100 токены для декодирования
    labels_for_decode = [t for t in labels_list if t != -100]
    labels_text_with_special = processor.tokenizer.decode(labels_for_decode, skip_special_tokens=False)
    print(f"\nТекст с спец токенами:\n{repr(labels_text_with_special)}")
    
    # Декодируем С skip_special_tokens
    labels_text = processor.tokenizer.decode(labels_for_decode, skip_special_tokens=True)
    print(f"\nТекст БЕЗ спец токенов:\n{labels_text}")
    
    # Reference из датасета (ключ "text")
    print(f"\nReference (оригинал):\n{sample['text']}")
    
    print("\n" + "="*80)


Анализ labels из датасета для sample #31

📋 LABELS (из датасета):

Количество токенов (включая -100): 12
Количество токенов без -100: 12

Первые 20 токенов: [50258, 50363, 8068, 39263, 4558, 8525, 12848, 757, 4953, 6790, 13, 50257]

Последние 20 токенов: [50258, 50363, 8068, 39263, 4558, 8525, 12848, 757, 4953, 6790, 13, 50257]

Текст с спец токенами:
'<|startoftranscript|><|notimestamps|>Спрятаться тут негде.<|endoftext|>'

Текст БЕЗ спец токенов:
Спрятаться тут негде.

Reference (оригинал):
Спрятаться тут негде.



In [5]:
# БЛОК 4: СРАВНЕНИЕ токенов из generate и labels

if perfect_sample_idx is None:
    print("⚠️ Пропускаем - sample не найден")
else:
    print("\n🔍 СРАВНЕНИЕ ТОКЕНОВ")
    print("="*80)
    
    # Получаем обе последовательности
    sample = train_dataset[perfect_sample_idx]
    
    # Generate tokens
    input_features = sample['input_features'].unsqueeze(0).to(device)
    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            language="ru",
            task="transcribe",
            max_length=448
        )
    predicted_tokens = predicted_ids[0].cpu().tolist()
    
    # Dataset labels
    labels = sample['labels'].cpu().tolist()
    labels_tokens = [t for t in labels if t != -100]  # Убираем -100
    
    print(f"\n📊 Статистика:")
    print(f"Generate tokens: {len(predicted_tokens)}")
    print(f"Labels tokens: {len(labels_tokens)}")
    print(f"Разница в длине: {abs(len(predicted_tokens) - len(labels_tokens))}")
    
    # Сравниваем первые N токенов
    print(f"\n🔍 Первые 30 токенов:")
    print(f"{'Позиция':<10} {'Generate':<15} {'Labels':<15} {'Совпадение'}")
    print("-" * 60)
    
    max_len = max(len(predicted_tokens), len(labels_tokens))
    differences = []
    
    for i in range(min(30, max_len)):
        pred_tok = predicted_tokens[i] if i < len(predicted_tokens) else "---"
        label_tok = labels_tokens[i] if i < len(labels_tokens) else "---"
        match = "✅" if pred_tok == label_tok else "❌"
        
        if pred_tok != label_tok:
            differences.append(i)
        
        print(f"{i:<10} {str(pred_tok):<15} {str(label_tok):<15} {match}")
    
    # Сравниваем последние N токенов
    print(f"\n🔍 Последние 20 токенов:")
    print(f"{'Позиция':<10} {'Generate':<15} {'Labels':<15} {'Совпадение'}")
    print("-" * 60)
    
    for i in range(max(0, max_len - 20), max_len):
        pred_tok = predicted_tokens[i] if i < len(predicted_tokens) else "---"
        label_tok = labels_tokens[i] if i < len(labels_tokens) else "---"
        match = "✅" if pred_tok == label_tok else "❌"
        
        if pred_tok != label_tok and i not in differences:
            differences.append(i)
        
        print(f"{i:<10} {str(pred_tok):<15} {str(label_tok):<15} {match}")
    
    # Декодируем различающиеся токены
    if differences:
        print(f"\n❌ НАЙДЕНО {len(differences[:10])} различий (показываем первые 10):")
        for diff_idx in differences[:10]:
            if diff_idx < len(predicted_tokens):
                pred_tok = predicted_tokens[diff_idx]
                pred_decoded = processor.tokenizer.decode([pred_tok])
                print(f"  Позиция {diff_idx}: Generate token {pred_tok} → '{pred_decoded}'")
            
            if diff_idx < len(labels_tokens):
                label_tok = labels_tokens[diff_idx]
                label_decoded = processor.tokenizer.decode([label_tok])
                print(f"  Позиция {diff_idx}: Labels token {label_tok} → '{label_decoded}'")
            print()
    else:
        print("\n✅ Все токены ИДЕНТИЧНЫ!")
    
    # Проверяем спец токены
    print(f"\n🎯 СПЕЦИАЛЬНЫЕ ТОКЕНЫ:")
    print(f"BOS token: {processor.tokenizer.bos_token} (id={processor.tokenizer.bos_token_id})")
    print(f"EOS token: {processor.tokenizer.eos_token} (id={processor.tokenizer.eos_token_id})")
    print(f"PAD token: {processor.tokenizer.pad_token} (id={processor.tokenizer.pad_token_id})")
    
    # Проверяем наличие спец токенов в обеих последовательностях
    print(f"\nВ Generate:")
    print(f"  BOS: {'✅' if processor.tokenizer.bos_token_id in predicted_tokens else '❌'}")
    print(f"  EOS: {'✅' if processor.tokenizer.eos_token_id in predicted_tokens else '❌'}")
    print(f"  PAD: {'✅' if processor.tokenizer.pad_token_id in predicted_tokens else '❌'}")
    
    print(f"\nВ Labels:")
    print(f"  BOS: {'✅' if processor.tokenizer.bos_token_id in labels_tokens else '❌'}")
    print(f"  EOS: {'✅' if processor.tokenizer.eos_token_id in labels_tokens else '❌'}")
    print(f"  PAD: {'✅' if processor.tokenizer.pad_token_id in labels_tokens else '❌'}")
    
    print("\n" + "="*80)


🔍 СРАВНЕНИЕ ТОКЕНОВ

📊 Статистика:
Generate tokens: 12
Labels tokens: 12
Разница в длине: 0

🔍 Первые 30 токенов:
Позиция    Generate        Labels          Совпадение
------------------------------------------------------------
0          2933            50258           ❌
1          1552            50363           ❌
2          1675            8068            ❌
3          681             39263           ❌
4          11508           4558            ❌
5          4396            8525            ❌
6          1814            12848           ❌
7          1414            757             ❌
8          1725            4953            ❌
9          17255           6790            ❌
10         3396            13              ❌
11         13              50257           ❌

🔍 Последние 20 токенов:
Позиция    Generate        Labels          Совпадение
------------------------------------------------------------
0          2933            50258           ❌
1          1552            50363           ❌


In [6]:
# БЛОК 5: Вычисление loss для этого sample

if perfect_sample_idx is None:
    print("⚠️ Пропускаем - sample не найден")
else:
    print("\n💰 ВЫЧИСЛЕНИЕ LOSS")
    print("="*80)
    
    # Получаем sample
    sample = train_dataset[perfect_sample_idx]
    input_features = sample['input_features'].unsqueeze(0).to(device)
    labels = sample['labels'].unsqueeze(0).to(device)
    
    # Forward pass с labels для получения loss
    print("Вычисляем loss через forward pass...")
    with torch.no_grad():
        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
    
    print(f"\n📊 РЕЗУЛЬТАТЫ:")
    print(f"Loss для этого sample: {loss.item():.4f}")
    print(f"Logits shape: {logits.shape}")  # (batch_size, sequence_length, vocab_size)
    
    # Проверим, что именно влияет на loss
    # Декодируем labels
    labels_cpu = labels[0].cpu().tolist()
    labels_no_ignore = [t for t in labels_cpu if t != -100]
    
    print(f"\n🔍 АНАЛИЗ:")
    print(f"Количество токенов для loss: {len(labels_no_ignore)}")
    
    # Посчитаем loss вручную для первых нескольких токенов
    import torch.nn.functional as F
    
    print(f"\n🎯 Loss по первым 10 токенам:")
    for i in range(min(10, len(labels_no_ignore))):
        token_id = labels_cpu[i]
        
        if token_id == -100:
            continue
            
        # Получаем логиты для этого токена
        token_logits = logits[0, i, :]  # shape: (vocab_size,)
        
        # Вычисляем вероятности
        probs = F.softmax(token_logits, dim=0)
        
        # Вероятность правильного токена
        correct_prob = probs[token_id].item()
        
        # Loss для этого токена: -log(P(correct_token))
        token_loss = -torch.log(probs[token_id]).item()
        
        # Декодируем токен
        token_text = processor.tokenizer.decode([token_id])
        
        print(f"  Токен {i}: id={token_id:5d} '{token_text:10s}' | P={correct_prob:.4f} | loss={token_loss:.4f}")
    
    print(f"\n✅ Если все токены идентичны, но loss высокий - значит модель не уверена в своих предсказаниях!")
    print(f"✅ WER может быть 0 (правильные слова), но loss высокий (низкая уверенность)")
    
    print("\n" + "="*80)


💰 ВЫЧИСЛЕНИЕ LOSS
Вычисляем loss через forward pass...

📊 РЕЗУЛЬТАТЫ:
Loss для этого sample: 9.0791
Logits shape: torch.Size([1, 12, 51865])

🔍 АНАЛИЗ:
Количество токенов для loss: 12

🎯 Loss по первым 10 токенам:
  Токен 0: id=50258 '<|startoftranscript|>' | P=0.0000 | loss=14.2814
  Токен 1: id=50363 '<|notimestamps|>' | P=0.0000 | loss=12.3371
  Токен 2: id= 8068 'С         ' | P=0.0000 | loss=13.1259
  Токен 3: id=39263 'пр        ' | P=0.0000 | loss=12.3315
  Токен 4: id= 4558 'ят        ' | P=0.0001 | loss=8.8245
  Токен 5: id= 8525 'аться     ' | P=0.0000 | loss=11.3015
  Токен 6: id=12848 ' тут      ' | P=0.0000 | loss=12.2148
  Токен 7: id=  757 ' н        ' | P=0.0109 | loss=4.5165
  Токен 8: id= 4953 'ег        ' | P=0.0003 | loss=8.0281
  Токен 9: id= 6790 'де        ' | P=0.0015 | loss=6.5041

✅ Если все токены идентичны, но loss высокий - значит модель не уверена в своих предсказаниях!
✅ WER может быть 0 (правильные слова), но loss высокий (низкая уверенность)



In [7]:
# БЛОК 6: Forward pass - проверка что модель предсказывает (teacher forcing)

if perfect_sample_idx is None:
    print("⚠️ Пропускаем - sample не найден")
else:
    print("\n🔄 FORWARD PASS (teacher forcing)")
    print("="*80)
    
    # Получаем sample
    sample = train_dataset[perfect_sample_idx]
    input_features = sample['input_features'].unsqueeze(0).to(device)
    labels = sample['labels'].unsqueeze(0).to(device)
    
    # Forward pass с labels (teacher forcing)
    print("Выполняем forward pass с labels...")
    with torch.no_grad():
        outputs = model(input_features=input_features, labels=labels)
        logits = outputs.logits  # (batch, sequence_length, vocab_size)
    
    # Предсказания модели (argmax)
    predicted_token_ids = torch.argmax(logits, dim=-1)[0]  # (sequence_length,)
    predicted_tokens = predicted_token_ids.cpu().tolist()
    
    # Labels
    labels_cpu = labels[0].cpu().tolist()
    labels_no_ignore = [t for t in labels_cpu if t != -100]
    
    print(f"\n📊 РЕЗУЛЬТАТЫ:")
    print(f"Labels length: {len(labels_no_ignore)}")
    print(f"Predicted length: {len(predicted_tokens)}")
    
    print(f"\n🔍 ПЕРВЫЕ 10 ТОКЕНОВ:")
    print(f"{'Позиция':<10} {'Labels':<15} {'Predicted':<15} {'Match':<10} {'Token'}")
    print("-" * 80)
    
    for i in range(min(10, len(labels_no_ignore))):
        label_tok = labels_cpu[i]
        if label_tok == -100:
            continue
        
        pred_tok = predicted_tokens[i]
        match = "✅" if label_tok == pred_tok else "❌"
        
        # Декодируем оба токена
        label_decoded = processor.tokenizer.decode([label_tok])
        pred_decoded = processor.tokenizer.decode([pred_tok])
        
        print(f"{i:<10} {label_tok:<15} {pred_tok:<15} {match:<10} L:'{label_decoded}' P:'{pred_decoded}'")
    
    # Декодируем полные последовательности
    print(f"\n📝 ПОЛНЫЕ ПОСЛЕДОВАТЕЛЬНОСТИ:")
    
    # Labels (без -100)
    labels_text = processor.tokenizer.decode(labels_no_ignore, skip_special_tokens=False)
    print(f"Labels: {repr(labels_text)}")
    
    # Predicted
    predicted_text = processor.tokenizer.decode(predicted_tokens, skip_special_tokens=False)
    print(f"Predicted: {repr(predicted_text)}")
    
    print("\n" + "="*80)


🔄 FORWARD PASS (teacher forcing)
Выполняем forward pass с labels...

📊 РЕЗУЛЬТАТЫ:
Labels length: 12
Predicted length: 12

🔍 ПЕРВЫЕ 10 ТОКЕНОВ:
Позиция    Labels          Predicted       Match      Token
--------------------------------------------------------------------------------
0          50258           50263           ❌          L:'<|startoftranscript|>' P:'<|ru|>'
1          50363           50263           ❌          L:'<|notimestamps|>' P:'<|ru|>'
2          8068            50257           ❌          L:'С' P:'<|endoftext|>'
3          39263           50257           ❌          L:'пр' P:'<|endoftext|>'
4          4558            39263           ❌          L:'ят' P:'пр'
5          8525            4558            ❌          L:'аться' P:'ят'
6          12848           50257           ❌          L:' тут' P:'<|endoftext|>'
7          757             510             ❌          L:' н' P:' here'
8          4953            757             ❌          L:'ег' P:' н'
9          6790      

In [8]:
# БЛОК 7: Проверка ПРАВИЛЬНЫХ спец токенов для русского языка

print("\n🎯 ПРАВИЛЬНЫЕ СПЕЦ ТОКЕНЫ ДЛЯ WHISPER")
print("="*80)

# Whisper tokenizer имеет специальные токены для языка и задачи
print("\n📋 Основные спец токены:")
print(f"<|startoftranscript|>: {processor.tokenizer.convert_tokens_to_ids('<|startoftranscript|>')}")
print(f"<|endoftext|>: {processor.tokenizer.eos_token_id}")

# Языковые токены
print(f"\n🌐 Языковые токены:")
print(f"<|ru|> (русский): {processor.tokenizer.convert_tokens_to_ids('<|ru|>')}")
print(f"<|en|> (английский): {processor.tokenizer.convert_tokens_to_ids('<|en|>')}")

# Задачные токены
print(f"\n📝 Задачные токены:")
print(f"<|transcribe|>: {processor.tokenizer.convert_tokens_to_ids('<|transcribe|>')}")
print(f"<|translate|>: {processor.tokenizer.convert_tokens_to_ids('<|translate|>')}")

# Timestamp токены
print(f"\n⏱️ Timestamp токены:")
print(f"<|notimestamps|>: {processor.tokenizer.convert_tokens_to_ids('<|notimestamps|>')}")

# Правильная последовательность для русского transcribe
print(f"\n✅ ПРАВИЛЬНАЯ ПОСЛЕДОВАТЕЛЬНОСТЬ для русского языка (transcribe, no timestamps):")
correct_prefix = [
    processor.tokenizer.convert_tokens_to_ids('<|startoftranscript|>'),
    processor.tokenizer.convert_tokens_to_ids('<|ru|>'),
    processor.tokenizer.convert_tokens_to_ids('<|transcribe|>'),
    processor.tokenizer.convert_tokens_to_ids('<|notimestamps|>')
]
print(f"Токены: {correct_prefix}")
for tok_id in correct_prefix:
    print(f"  {tok_id}: {processor.tokenizer.decode([tok_id])}")

# Что есть в наших labels
print(f"\n❌ ЧТО СЕЙЧАС В LABELS (sample #{perfect_sample_idx}):")
sample = train_dataset[perfect_sample_idx]
labels_list = sample['labels'].cpu().tolist()
labels_first_4 = labels_list[:4]
print(f"Первые 4 токена: {labels_first_4}")
for tok_id in labels_first_4:
    if tok_id != -100:
        print(f"  {tok_id}: {processor.tokenizer.decode([tok_id])}")

print(f"\n🔍 СРАВНЕНИЕ:")
if labels_first_4[:4] == correct_prefix:
    print("✅ Labels содержат ВСЕ необходимые префикс токены!")
else:
    print("❌ Labels НЕ содержат все префикс токены!")
    print(f"Ожидалось: {correct_prefix}")
    print(f"Получено: {labels_first_4}")

print("\n" + "="*80)


🎯 ПРАВИЛЬНЫЕ СПЕЦ ТОКЕНЫ ДЛЯ WHISPER

📋 Основные спец токены:
<|startoftranscript|>: 50258
<|endoftext|>: 50257

🌐 Языковые токены:
<|ru|> (русский): 50263
<|en|> (английский): 50259

📝 Задачные токены:
<|transcribe|>: 50359
<|translate|>: 50358

⏱️ Timestamp токены:
<|notimestamps|>: 50363

✅ ПРАВИЛЬНАЯ ПОСЛЕДОВАТЕЛЬНОСТЬ для русского языка (transcribe, no timestamps):
Токены: [50258, 50263, 50359, 50363]
  50258: <|startoftranscript|>
  50263: <|ru|>
  50359: <|transcribe|>
  50363: <|notimestamps|>

❌ ЧТО СЕЙЧАС В LABELS (sample #31):
Первые 4 токена: [50258, 50363, 8068, 39263]
  50258: <|startoftranscript|>
  50363: <|notimestamps|>
  8068: С
  39263: пр

🔍 СРАВНЕНИЕ:
❌ Labels НЕ содержат все префикс токены!
Ожидалось: [50258, 50263, 50359, 50363]
Получено: [50258, 50363, 8068, 39263]



In [9]:
# БЛОК 8: Проверка атрибутов токенизатора и способов вызова

print("\n🔍 АТРИБУТЫ ТОКЕНИЗАТОРА")
print("="*80)

# Проверим processor из датасета
print("\n📋 Processor и Tokenizer:")
print(f"Type processor: {type(processor)}")
print(f"Type tokenizer: {type(processor.tokenizer)}")

# Ключевые атрибуты
print(f"\n🌐 Языковые настройки:")
print(f"tokenizer.language: {getattr(processor.tokenizer, 'language', 'NOT SET')}")
print(f"tokenizer.task: {getattr(processor.tokenizer, 'task', 'NOT SET')}")
print(f"tokenizer.predict_timestamps: {getattr(processor.tokenizer, 'predict_timestamps', 'NOT SET')}")

# Проверим prefix_tokens
if hasattr(processor.tokenizer, 'prefix_tokens'):
    prefix = processor.tokenizer.prefix_tokens
    print(f"\n✅ prefix_tokens: {prefix}")
    print("Декодировано:")
    for tok_id in prefix:
        print(f"  {tok_id}: {processor.tokenizer.decode([tok_id])}")
else:
    print("\n❌ prefix_tokens НЕ СУЩЕСТВУЕТ!")

# Тест токенизации
print(f"\n🧪 ТЕСТ ТОКЕНИЗАЦИИ (текущий processor):")
test_text = "Она удивительная женщина."
test_tokens = processor.tokenizer(test_text, return_tensors="pt").input_ids[0]
print(f"Текст: {test_text}")
print(f"Токены: {test_tokens.tolist()}")
print(f"Декодировано: {repr(processor.tokenizer.decode(test_tokens, skip_special_tokens=False))}")

# Проверим, какой токенизатор используется в датасете
print(f"\n📦 Токенизатор в датасете:")
dataset_tokenizer = train_dataset.tokenizer
print(f"Type: {type(dataset_tokenizer)}")
print(f"language: {getattr(dataset_tokenizer, 'language', 'NOT SET')}")
print(f"task: {getattr(dataset_tokenizer, 'task', 'NOT SET')}")

if hasattr(dataset_tokenizer, 'prefix_tokens'):
    prefix_dataset = dataset_tokenizer.prefix_tokens
    print(f"prefix_tokens: {prefix_dataset}")
else:
    print("❌ prefix_tokens НЕ СУЩЕСТВУЕТ!")

# ========== НОВЫЙ ТЕСТ ==========
print("\n" + "="*80)
print("🧪 СОЗДАНИЕ НОВОГО ТОКЕНИЗАТОРА С ЯВНЫМИ language И task")
print("="*80)

from transformers import WhisperTokenizer

# Создаем новый токенизатор с language и task
print("\n1️⃣ Создание токенизатора:")
tokenizer_with_lang = WhisperTokenizer.from_pretrained(
    "openai/whisper-base",
    language="ru",
    task="transcribe"
)
print(f"✅ Токенизатор создан")
print(f"   language: {getattr(tokenizer_with_lang, 'language', 'NOT SET')}")
print(f"   task: {getattr(tokenizer_with_lang, 'task', 'NOT SET')}")
print(f"   predict_timestamps: {getattr(tokenizer_with_lang, 'predict_timestamps', 'NOT SET')}")

if hasattr(tokenizer_with_lang, 'prefix_tokens'):
    prefix_new = tokenizer_with_lang.prefix_tokens
    print(f"   prefix_tokens: {prefix_new}")
    for tok_id in prefix_new:
        print(f"     {tok_id}: {tokenizer_with_lang.decode([tok_id])}")

# Вариант 1: Вызов БЕЗ явного указания language/task (как в __getitem__)
print("\n2️⃣ ВАРИАНТ 1: Вызов БЕЗ явного указания language/task (как в __getitem__):")
tokens_variant1 = tokenizer_with_lang(test_text, return_tensors="pt").input_ids[0]
print(f"Токены: {tokens_variant1.tolist()}")
decoded1 = tokenizer_with_lang.decode(tokens_variant1, skip_special_tokens=False)
print(f"Декодировано: {repr(decoded1)}")
print(f"Первые 4 токена: {tokens_variant1[:4].tolist()}")

# Вариант 2: Вызов С ЯВНЫМ указанием language/task
print("\n3️⃣ ВАРИАНТ 2: Вызов С ЯВНЫМ указанием language/task:")
# Проверим, принимает ли __call__ эти параметры
try:
    tokens_variant2 = tokenizer_with_lang(
        test_text,
        return_tensors="pt",
        language="ru",
        task="transcribe"
    ).input_ids[0]
    print(f"Токены: {tokens_variant2.tolist()}")
    decoded2 = tokenizer_with_lang.decode(tokens_variant2, skip_special_tokens=False)
    print(f"Декодировано: {repr(decoded2)}")
    print(f"Первые 4 токена: {tokens_variant2[:4].tolist()}")
except TypeError as e:
    print(f"❌ Ошибка: {e}")
    print("Токенизатор НЕ принимает language/task в __call__")

# Сравнение
print("\n4️⃣ СРАВНЕНИЕ:")
expected_prefix = [50258, 50263, 50359, 50363]  # <|startoftranscript|> <|ru|> <|transcribe|> <|notimestamps|>
actual_prefix = tokens_variant1[:4].tolist()

if actual_prefix == expected_prefix:
    print("✅ Вариант 1 дает ПРАВИЛЬНЫЕ префикс токены!")
else:
    print("❌ Вариант 1 дает НЕПРАВИЛЬНЫЕ префикс токены!")
    print(f"   Ожидалось: {expected_prefix}")
    print(f"   Получено:  {actual_prefix}")

print("\n" + "="*80)


🔍 АТРИБУТЫ ТОКЕНИЗАТОРА

📋 Processor и Tokenizer:
Type processor: <class 'transformers.models.whisper.processing_whisper.WhisperProcessor'>
Type tokenizer: <class 'transformers.models.whisper.tokenization_whisper_fast.WhisperTokenizerFast'>

🌐 Языковые настройки:
tokenizer.language: None
tokenizer.task: None
tokenizer.predict_timestamps: False

✅ prefix_tokens: [50258, 50363]
Декодировано:
  50258: <|startoftranscript|>
  50363: <|notimestamps|>

🧪 ТЕСТ ТОКЕНИЗАЦИИ (текущий processor):
Текст: Она удивительная женщина.
Токены: [50258, 50363, 7876, 1931, 36459, 7809, 6323, 28393, 7861, 13, 50257]
Декодировано: '<|startoftranscript|><|notimestamps|>Она удивительная женщина.<|endoftext|>'

📦 Токенизатор в датасете:
Type: <class 'transformers.models.whisper.tokenization_whisper_fast.WhisperTokenizerFast'>
language: None
task: None
prefix_tokens: [50258, 50363]

🧪 СОЗДАНИЕ НОВОГО ТОКЕНИЗАТОРА С ЯВНЫМИ language И task

1️⃣ Создание токенизатора:


Keyword arguments {'language': 'ru', 'task': 'transcribe'} not recognized.


✅ Токенизатор создан
   language: ru
   task: transcribe
   predict_timestamps: False
   prefix_tokens: [50258, 50263, 50359, 50363]
     50258: <|startoftranscript|>
     50263: <|ru|>
     50359: <|transcribe|>
     50363: <|notimestamps|>

2️⃣ ВАРИАНТ 1: Вызов БЕЗ явного указания language/task (как в __getitem__):
Токены: [50258, 50263, 50359, 50363, 7876, 1931, 36459, 7809, 6323, 28393, 7861, 13, 50257]
Декодировано: '<|startoftranscript|><|ru|><|transcribe|><|notimestamps|>Она удивительная женщина.<|endoftext|>'
Первые 4 токена: [50258, 50263, 50359, 50363]

3️⃣ ВАРИАНТ 2: Вызов С ЯВНЫМ указанием language/task:
Токены: [50258, 50263, 50359, 50363, 7876, 1931, 36459, 7809, 6323, 28393, 7861, 13, 50257]
Декодировано: '<|startoftranscript|><|ru|><|transcribe|><|notimestamps|>Она удивительная женщина.<|endoftext|>'
Первые 4 токена: [50258, 50263, 50359, 50363]

4️⃣ СРАВНЕНИЕ:
✅ Вариант 1 дает ПРАВИЛЬНЫЕ префикс токены!



In [ ]:
# БЛОК 9: Проверка setup_processor() из DataManager

print("\n🧪 ПРОВЕРКА DataManager.setup_processor()")
print("="*80)

# Создаем НОВЫЙ DataManager (не используем старый из блока 1)
print("\n1️⃣ Создание нового DataManager:")
from src.data import DataManager
from src.config import load_config

# Загружаем конфиг
new_config = load_config("../configs/whisper_base.yaml")
print(f"✅ Конфиг загружен")
print(f"   data.language: {new_config.data.language}")
print(f"   data.task: {new_config.data.task}")

# Создаем DataManager
new_data_manager = DataManager(new_config)
print(f"✅ DataManager создан")

# Вызываем setup_processor() с ЯВНЫМИ параметрами language и task
print("\n2️⃣ Вызов setup_processor() с явными параметрами:")
new_data_manager.setup_processor(
    model_name="openai/whisper-base",
    model_type="whisper",
    language=new_config.data.language,
    task=new_config.data.task
)
print(f"✅ setup_processor() выполнен с language='{new_config.data.language}', task='{new_config.data.task}'")

# Проверяем созданный processor
new_processor = new_data_manager.processor
new_tokenizer = new_processor.tokenizer

print("\n3️⃣ ПРОВЕРКА АТРИБУТОВ ТОКЕНИЗАТОРА:")
print(f"Type processor: {type(new_processor)}")
print(f"Type tokenizer: {type(new_tokenizer)}")
print(f"\n🌐 Языковые настройки:")
print(f"   language: {getattr(new_tokenizer, 'language', 'NOT SET')}")
print(f"   task: {getattr(new_tokenizer, 'task', 'NOT SET')}")
print(f"   predict_timestamps: {getattr(new_tokenizer, 'predict_timestamps', 'NOT SET')}")

if hasattr(new_tokenizer, 'prefix_tokens'):
    new_prefix = new_tokenizer.prefix_tokens
    print(f"\n✅ prefix_tokens: {new_prefix}")
    print("   Декодировано:")
    for tok_id in new_prefix:
        print(f"     {tok_id}: {new_tokenizer.decode([tok_id])}")
else:
    print("\n❌ prefix_tokens НЕ СУЩЕСТВУЕТ!")

# Проверяем правильность префикса
expected_prefix = [50258, 50263, 50359, 50363]  # <|startoftranscript|> <|ru|> <|transcribe|> <|notimestamps|>
if hasattr(new_tokenizer, 'prefix_tokens'):
    actual_prefix = new_tokenizer.prefix_tokens
    if actual_prefix == expected_prefix:
        print("\n✅✅✅ ПРЕФИКС ТОКЕНЫ ПРАВИЛЬНЫЕ!")
    else:
        print("\n❌❌❌ ПРЕФИКС ТОКЕНЫ НЕПРАВИЛЬНЫЕ!")
        print(f"   Ожидалось: {expected_prefix}")
        print(f"   Получено:  {actual_prefix}")

# Создаем dataset с НОВЫМ processor
print("\n4️⃣ Создание датасета с новым processor:")
new_train_dataset = new_data_manager.create_dataset('train')
print(f"✅ Датасет создан: {len(new_train_dataset)} samples")

# Берём один сэмпл
print("\n5️⃣ Проверка labels из датасета:")
test_sample = new_train_dataset[0]
test_labels = test_sample['labels'].cpu().tolist()
test_labels_first_4 = test_labels[:4]

print(f"Первые 4 токена из labels: {test_labels_first_4}")
print("Декодировано:")
for tok_id in test_labels_first_4:
    if tok_id != -100:
        print(f"  {tok_id}: {new_tokenizer.decode([tok_id])}")

# Финальная проверка
print("\n6️⃣ ФИНАЛЬНАЯ ПРОВЕРКА:")
if test_labels_first_4 == expected_prefix:
    print("✅✅✅ LABELS СОДЕРЖАТ ВСЕ 4 ПРЕФИКС ТОКЕНА!")
    print("     setup_processor() РАБОТАЕТ ПРАВИЛЬНО!")
else:
    print("❌❌❌ LABELS НЕ СОДЕРЖАТ ВСЕ ПРЕФИКС ТОКЕНЫ!")
    print(f"     Ожидалось: {expected_prefix}")
    print(f"     Получено:  {test_labels_first_4}")

print("\n" + "="*80)

In [4]:
# БЛОК 10: Forward pass для батча из 8 аудио с правильным processor (самодостаточный блок)

print("\n🚀 FORWARD PASS ДЛЯ БАТЧА ИЗ 8 АУДИО (с правильным processor)")
print("="*80)

import sys
sys.path.append('..')

import torch
from transformers import WhisperForConditionalGeneration
from src.data import DataManager, AudioCollator
from src.config import load_config
from torch.utils.data import DataLoader
import torch.nn.functional as F

# 1️⃣ Загрузка конфигурации и создание DataManager
print("\n1️⃣ Инициализация окружения:")
config = load_config("../configs/whisper_base.yaml")
print(f"✅ Конфиг загружен: language={config.data.language}, task={config.data.task}")

# Создаем DataManager
data_manager = DataManager(config)
print(f"✅ DataManager создан")

# Вызываем setup_processor() с ЯВНЫМИ параметрами language и task
data_manager.setup_processor(
    model_name="openai/whisper-base",
    model_type="whisper",
    language=config.data.language,
    task=config.data.task
)
print(f"✅ Processor создан с language='{config.data.language}', task='{config.data.task}'")

# Проверяем prefix_tokens
processor = data_manager.processor
tokenizer = processor.tokenizer
if hasattr(tokenizer, 'prefix_tokens'):
    print(f"   prefix_tokens: {tokenizer.prefix_tokens}")
    expected = [50258, 50263, 50359, 50363]
    if tokenizer.prefix_tokens == expected:
        print(f"   ✅ Префикс токены ПРАВИЛЬНЫЕ!")
    else:
        print(f"   ❌ ОШИБКА: ожидалось {expected}")

# 2️⃣ Загрузка модели
print("\n2️⃣ Загрузка модели:")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
model = model.to(device)
model.eval()
print(f"✅ Модель загружена на {device}")

# 3️⃣ Создание датасета и DataLoader
print("\n3️⃣ Создание датасета:")
train_dataset = data_manager.create_dataset('train')
print(f"✅ Датасет создан: {len(train_dataset)} samples")

# Создаем collator (принимает config, model_type, transpose_features)
collator = AudioCollator(
    config=config.data,
    model_type="whisper",
    transpose_features=False
)

# Создаем DataLoader с batch_size=8
dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collator,
    num_workers=0
)

# Получаем первый батч
batch = next(iter(dataloader))
input_features = batch['input_features'].to(device)
labels = batch['labels'].to(device)
references = batch['text']

print(f"✅ Батч загружен")
print(f"   Batch size: {input_features.shape[0]}")
print(f"   Input features shape: {input_features.shape}")
print(f"   Labels shape: {labels.shape}")

# 4️⃣ Forward pass для всего батча
print("\n4️⃣ Forward pass для батча:")
with torch.no_grad():
    outputs = model(input_features=input_features, labels=labels)
    batch_loss = outputs.loss
    logits = outputs.logits

print(f"✅ Forward pass завершён")
print(f"   Средний loss по батчу (автоматический): {batch_loss.item():.4f}")
print(f"   Logits shape: {logits.shape}")

# 5️⃣ Вычисление loss вручную тремя способами
print("\n5️⃣ ВЫЧИСЛЕНИЕ LOSS ВРУЧНУЮ ТРЕМЯ СПОСОБАМИ:")
print("="*80)

# Вычисляем loss вручную для всего батча
def compute_manual_loss(logits, labels, ignore_first_n=0):
    """
    Вычисляет cross-entropy loss вручную, игнорируя первые N токенов.
    
    Args:
        logits: (batch_size, seq_len, vocab_size)
        labels: (batch_size, seq_len)
        ignore_first_n: Количество первых токенов для игнорирования (0, 1, или 4)
    
    Returns:
        average loss (скаляр)
    """
    batch_size, seq_len, vocab_size = logits.shape
    
    # Создаем маску для игнорирования первых N токенов
    mask = torch.ones_like(labels, dtype=torch.bool)
    if ignore_first_n > 0:
        mask[:, :ignore_first_n] = False
    
    # Также игнорируем -100 токены
    mask = mask & (labels != -100)
    
    # Flatten логиты и labels
    logits_flat = logits.view(-1, vocab_size)  # (batch*seq_len, vocab_size)
    labels_flat = labels.view(-1)  # (batch*seq_len,)
    mask_flat = mask.view(-1)  # (batch*seq_len,)
    
    # Вычисляем cross-entropy для каждой позиции
    loss_per_token = F.cross_entropy(logits_flat, labels_flat, reduction='none')  # (batch*seq_len,)
    
    # Применяем маску и вычисляем среднее
    masked_loss = loss_per_token * mask_flat.float()
    total_loss = masked_loss.sum()
    num_tokens = mask_flat.sum()
    
    if num_tokens > 0:
        avg_loss = total_loss / num_tokens
    else:
        avg_loss = torch.tensor(0.0)
    
    return avg_loss.item(), num_tokens.item()

# Вариант 1: С учетом всех токенов (включая первые 4)
loss_with_all, tokens_with_all = compute_manual_loss(logits, labels, ignore_first_n=0)
print(f"\n1️⃣ Loss С УЧЕТОМ первых 4 токенов:")
print(f"   Loss: {loss_with_all:.4f}")
print(f"   Токенов учтено: {tokens_with_all}")

# Вариант 2: БЕЗ учета первых 4 токенов
loss_without_4, tokens_without_4 = compute_manual_loss(logits, labels, ignore_first_n=4)
print(f"\n2️⃣ Loss БЕЗ УЧЕТА первых 4 токенов:")
print(f"   Loss: {loss_without_4:.4f}")
print(f"   Токенов учтено: {tokens_without_4}")
print(f"   Разница с вариантом 1: {loss_with_all - loss_without_4:.4f}")

# Вариант 3: БЕЗ учета только первого токена
loss_without_1, tokens_without_1 = compute_manual_loss(logits, labels, ignore_first_n=1)
print(f"\n3️⃣ Loss БЕЗ УЧЕТА только первого токена:")
print(f"   Loss: {loss_without_1:.4f}")
print(f"   Токенов учтено: {tokens_without_1}")
print(f"   Разница с вариантом 1: {loss_with_all - loss_without_1:.4f}")

print(f"\n📊 СРАВНЕНИЕ:")
print(f"   Автоматический loss (PyTorch): {batch_loss.item():.4f}")
print(f"   Ручной loss (все токены):      {loss_with_all:.4f}")
print(f"   Ручной loss (без 4 префиксов): {loss_without_4:.4f}")
print(f"   Ручной loss (без 1 префикса):  {loss_without_1:.4f}")

# 6️⃣ Детальный анализ по каждому примеру
print("\n6️⃣ ДЕТАЛЬНЫЙ АНАЛИЗ КАЖДОГО ПРИМЕРА:")
print("="*80)

for idx in range(input_features.shape[0]):
    print(f"\n{'='*80}")
    print(f"ПРИМЕР {idx + 1}/8")
    print(f"{'='*80}")
    
    # Данные для одного примера
    example_input = input_features[idx:idx+1]
    example_labels = labels[idx:idx+1]
    example_reference = references[idx]
    example_logits = logits[idx:idx+1]
    
    # Вычисляем loss для этого примера
    with torch.no_grad():
        example_output = model(input_features=example_input, labels=example_labels)
        example_loss_auto = example_output.loss
    
    # Loss вручную тремя способами для этого примера
    loss1, tok1 = compute_manual_loss(example_logits, example_labels, ignore_first_n=0)
    loss2, tok2 = compute_manual_loss(example_logits, example_labels, ignore_first_n=4)
    loss3, tok3 = compute_manual_loss(example_logits, example_labels, ignore_first_n=1)
    
    print(f"\n💰 LOSS:")
    print(f"   Автоматический:           {example_loss_auto.item():.4f}")
    print(f"   Ручной (все токены):      {loss1:.4f}")
    print(f"   Ручной (без 4 префиксов): {loss2:.4f} (токенов: {tok2})")
    print(f"   Ручной (без 1 префикса):  {loss3:.4f} (токенов: {tok3})")
    print(f"   Вклад 4 префиксов:        {loss1 - loss2:.4f}")
    print(f"   Вклад 1 префикса:         {loss1 - loss3:.4f}")
    
    # Предсказанные токены (argmax)
    predicted_token_ids = torch.argmax(example_logits.squeeze(0), dim=-1)
    predicted_tokens = predicted_token_ids.cpu().tolist()
    
    # Labels (убираем -100)
    labels_list = example_labels.squeeze(0).cpu().tolist()
    labels_clean = [t for t in labels_list if t != -100]

    print(f"\n📝 ЛЕЙБЛЫ:")
    print(f"   Reference:  {example_reference}")
    print(f"   Labels:     {labels_clean}")
    print(f"   Predicted:  {predicted_tokens}")

    # Декодируем в текст
    predicted_text = tokenizer.decode(predicted_tokens, skip_special_tokens=True)
    labels_text = tokenizer.decode(labels_clean, skip_special_tokens=True)
    
    print(f"\n📝 ТЕКСТЫ:")
    print(f"   Reference:  {example_reference}")
    print(f"   Labels:     {labels_text}")
    print(f"   Predicted:  {predicted_text}")
    
    # Loss по префикс токенам (первые 4)
    print(f"\n💡 LOSS ПО ПРЕФИКС ТОКЕНАМ (первые 4):")
    for i in range(min(4, len(labels_clean))):
        token_id = labels_clean[i]
        token_logits = example_logits.squeeze(0)[i]
        
        probs = F.softmax(token_logits, dim=0)
        correct_prob = probs[token_id].item()
        token_loss = -torch.log(probs[token_id]).item()
        
        token_text = tokenizer.decode([token_id])
        print(f"   Токен {i}: id={token_id:5d} '{token_text:20s}' | P={correct_prob:.6f} | loss={token_loss:.4f}")

# 7️⃣ Итоговая статистика
print(f"\n{'='*80}")
print("✅ АНАЛИЗ ЗАВЕРШЁН!")
print(f"{'='*80}")

print("\n📊 ИТОГОВАЯ СТАТИСТИКА ПО БАТЧУ:")
print(f"   Автоматический loss:           {batch_loss.item():.4f}")
print(f"   Ручной loss (все токены):      {loss_with_all:.4f}")
print(f"   Ручной loss (без 4 префиксов): {loss_without_4:.4f}")
print(f"   Ручной loss (без 1 префикса):  {loss_without_1:.4f}")
print(f"\n   Вклад 4 префикс токенов в loss: {loss_with_all - loss_without_4:.4f}")
print(f"   Вклад 1 префикс токена в loss:  {loss_with_all - loss_without_1:.4f}")
print(f"\n   Ожидаемый loss (после исправления): ~3-4")
print(f"   Старый loss (с багом): >12.0")

if batch_loss.item() < 5.0:
    print("\n✅✅✅ LOSS ЗНАЧИТЕЛЬНО СНИЗИЛСЯ!")
    print("     Исправление бага с prefix_tokens СРАБОТАЛО!")
else:
    print("\n⚠️ Loss всё ещё высокий, возможно есть другие проблемы")

print("\n" + "="*80)


🚀 FORWARD PASS ДЛЯ БАТЧА ИЗ 8 АУДИО (с правильным processor)

1️⃣ Инициализация окружения:
✅ Конфиг загружен: language=ru, task=transcribe
✅ DataManager создан
✅ Processor создан с language='ru', task='transcribe'
   prefix_tokens: [50258, 50263, 50359, 50363]
   ✅ Префикс токены ПРАВИЛЬНЫЕ!

2️⃣ Загрузка модели:
✅ Модель загружена на cuda

3️⃣ Создание датасета:
✅ Датасет создан: 26654 samples
✅ Батч загружен
   Batch size: 8
   Input features shape: torch.Size([8, 80, 3000])
   Labels shape: torch.Size([8, 27])

4️⃣ Forward pass для батча:
✅ Forward pass завершён
   Средний loss по батчу (автоматический): 2.9804
   Logits shape: torch.Size([8, 27, 51865])

5️⃣ ВЫЧИСЛЕНИЕ LOSS ВРУЧНУЮ ТРЕМЯ СПОСОБАМИ:

1️⃣ Loss С УЧЕТОМ первых 4 токенов:
   Loss: 2.9804
   Токенов учтено: 125

2️⃣ Loss БЕЗ УЧЕТА первых 4 токенов:
   Loss: 2.2322
   Токенов учтено: 93
   Разница с вариантом 1: 0.7482

3️⃣ Loss БЕЗ УЧЕТА только первого токена:
   Loss: 1.9508
   Токенов учтено: 117
   Разница с вариант

In [1]:
# БЛОК 11: Forward pass с моделью, загруженной через ModelManager.load_checkpoint() (как в train.py)

print("\n🚀 FORWARD PASS С МОДЕЛЬЮ ИЗ CHECKPOINT (загрузка как в train.py)")
print("="*80)

import sys
sys.path.append('..')

import torch
from src.models import ModelManager
from src.data import DataManager, AudioCollator
from src.config import load_config
from torch.utils.data import DataLoader
import torch.nn.functional as F

# 1️⃣ Загрузка конфигурации
print("\n1️⃣ Инициализация окружения:")
config = load_config("../configs/whisper_base.yaml")
print(f"✅ Конфиг загружен: language={config.data.language}, task={config.data.task}")

# 2️⃣ Загрузка модели через ModelManager (КАК В TRAIN.PY!)
print("\n2️⃣ Загрузка модели через ModelManager.load_checkpoint():")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

model_manager = ModelManager()
baseline_path = "../experiments/baselines/whisper-base"

# ВАЖНО: Загружаем модель ТОЧНО так же, как в train.py
model, processor, checkpoint_info = model_manager.load_checkpoint(
    baseline_path,
    device=device,
    processor=None,  # Will be auto-created from checkpoint model_metadata
    compile=False,
    language=config.data.language,  # Required for Whisper processor creation
    task=config.data.task  # Required for Whisper processor creation
)
model.eval()
print(f"✅ Модель загружена из {baseline_path}")
print(f"   Model name: {checkpoint_info.get('model_name', 'unknown')}")
print(f"   Epoch: {checkpoint_info.get('epoch', 'None (baseline)')}")

# Проверяем prefix_tokens
tokenizer = processor.tokenizer
if hasattr(tokenizer, 'prefix_tokens'):
    print(f"\n🔍 Processor prefix_tokens: {tokenizer.prefix_tokens}")
    expected = [50258, 50263, 50359, 50363]
    if tokenizer.prefix_tokens == expected:
        print(f"   ✅ Префикс токены ПРАВИЛЬНЫЕ!")
    else:
        print(f"   ❌ ОШИБКА: ожидалось {expected}")
        print(f"   Получено: {tokenizer.prefix_tokens}")
else:
    print("   ❌ prefix_tokens НЕ СУЩЕСТВУЕТ!")

# 3️⃣ Создание DataManager и датасета
print("\n3️⃣ Создание датасета:")
data_manager = DataManager(config)
data_manager.set_already_loaded_processor(processor)  # Используем processor от модели
print(f"✅ DataManager создан с processor от модели")

train_dataset = data_manager.create_dataset('train')
print(f"✅ Датасет создан: {len(train_dataset)} samples")

# Проверяем prefix_tokens в датасете
dataset_tokenizer = train_dataset.tokenizer
if hasattr(dataset_tokenizer, 'prefix_tokens'):
    print(f"   Датасет prefix_tokens: {dataset_tokenizer.prefix_tokens}")

# Создаем collator и DataLoader
collator = AudioCollator(
    config=config.data,
    model_type="whisper",
    transpose_features=False
)

dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collator,
    num_workers=0
)

# Получаем первый батч
batch = next(iter(dataloader))
input_features = batch['input_features'].to(device)
labels = batch['labels'].to(device)
references = batch['text']

print(f"✅ Батч загружен")
print(f"   Batch size: {input_features.shape[0]}")
print(f"   Input features shape: {input_features.shape}")
print(f"   Labels shape: {labels.shape}")

# 4️⃣ Forward pass для всего батча
print("\n4️⃣ Forward pass для батча:")
with torch.no_grad():
    outputs = model(input_features=input_features, labels=labels)
    batch_loss = outputs.loss
    logits = outputs.logits

print(f"✅ Forward pass завершён")
print(f"   Средний loss по батчу: {batch_loss.item():.4f}")
print(f"   Logits shape: {logits.shape}")

# 5️⃣ Детальный анализ первых 3 примеров (как в train.py DEBUG)
print("\n5️⃣ ДЕТАЛЬНЫЙ АНАЛИЗ ПЕРВЫХ 3 ПРИМЕРОВ:")
print("="*80)

for idx in range(min(3, input_features.shape[0])):
    print(f"\n{'='*80}")
    print(f"ПРИМЕР {idx + 1}/{input_features.shape[0]}")
    print(f"{'='*80}")
    
    example_labels = labels[idx].cpu().tolist()
    example_labels_clean = [t for t in example_labels if t != -100]
    
    # Check prefix tokens
    print(f"\n🔍 PREFIX TOKENS:")
    print(f"First 4 tokens in labels: {example_labels_clean[:4]}")
    expected_prefix = [50258, 50263, 50359, 50363]
    if example_labels_clean[:4] == expected_prefix:
        print("✅ Labels contain CORRECT 4 prefix tokens")
    else:
        print(f"❌ WRONG prefix tokens!")
        print(f"   Expected: {expected_prefix}")
        print(f"   Got:      {example_labels_clean[:4]}")
    
    # Predicted tokens (argmax)
    example_logits = logits[idx]
    predicted_token_ids = torch.argmax(example_logits, dim=-1).cpu().tolist()
    
    print(f"\n📊 TOKEN COUNTS:")
    print(f"Labels tokens:    {len(example_labels_clean)}")
    print(f"Predicted tokens: {len(predicted_token_ids)}")
    
    # Decode to text
    reference_text = references[idx]
    labels_text = tokenizer.decode(example_labels_clean, skip_special_tokens=True)
    predicted_text = tokenizer.decode(predicted_token_ids, skip_special_tokens=True)
    
    print(f"\n📝 TEXTS:")
    print(f"Reference:  {reference_text}")
    print(f"Labels:     {labels_text}")
    print(f"Predicted:  {predicted_text}")
    
    # Loss by first 10 tokens
    print(f"\n💡 LOSS PER TOKEN (first 10):")
    for i in range(min(10, len(example_labels_clean))):
        token_id = example_labels_clean[i]
        if i < len(example_logits):
            token_logits = example_logits[i]
            probs = F.softmax(token_logits, dim=0)
            correct_prob = probs[token_id].item()
            token_loss = -torch.log(probs[token_id]).item()
            token_text = tokenizer.decode([token_id])
            print(f"  Token {i}: id={token_id:5d} '{token_text:20s}' | P={correct_prob:.6f} | loss={token_loss:.4f}")

# 6️⃣ Итоговая статистика
print(f"\n{'='*80}")
print("✅ АНАЛИЗ ЗАВЕРШЁН!")
print(f"{'='*80}")

print(f"\n📊 ИТОГОВАЯ СТАТИСТИКА:")
print(f"   Batch loss: {batch_loss.item():.4f}")
print(f"\n   ❓ ВОПРОС: Почему loss ~11-12, хотя processor ПРАВИЛЬНЫЙ (4 prefix_tokens)?")
print(f"\n   💡 ВОЗМОЖНАЯ ПРИЧИНА:")
print(f"      Модель openai/whisper-base обучалась с АВТОМАТИЧЕСКИМ определением языка!")
print(f"      Она ожидает САМА выбрать язык, а мы форсируем <|ru|> на позиции 1.")
print(f"      Это вызывает высокий loss на prefix токенах, потому что модель")
print(f"      'не понимает', почему язык зафиксирован.")
print(f"\n   🎯 СЛЕДУЮЩИЙ ШАГ:")
print(f"      Проверить официальные примеры Whisper fine-tuning - какой формат labels")
print(f"      используется там? Включают ли они <|ru|> и <|transcribe|> или нет?")

print("\n" + "="*80)


🚀 FORWARD PASS С МОДЕЛЬЮ ИЗ CHECKPOINT (загрузка как в train.py)

1️⃣ Инициализация окружения:
✅ Конфиг загружен: language=ru, task=transcribe

2️⃣ Загрузка модели через ModelManager.load_checkpoint():
Device: cuda
✅ Модель загружена из ../experiments/baselines/whisper-base
   Model name: openai/whisper-base
   Epoch: None

🔍 Processor prefix_tokens: [50258, 50263, 50359, 50363]
   ✅ Префикс токены ПРАВИЛЬНЫЕ!

3️⃣ Создание датасета:
✅ DataManager создан с processor от модели
✅ Датасет создан: 26654 samples
   Датасет prefix_tokens: [50258, 50263, 50359, 50363]
✅ Батч загружен
   Batch size: 8
   Input features shape: torch.Size([8, 80, 3000])
   Labels shape: torch.Size([8, 27])

4️⃣ Forward pass для батча:
✅ Forward pass завершён
   Средний loss по батчу: 3.2272
   Logits shape: torch.Size([8, 27, 51865])

5️⃣ ДЕТАЛЬНЫЙ АНАЛИЗ ПЕРВЫХ 3 ПРИМЕРОВ:

ПРИМЕР 1/8

🔍 PREFIX TOKENS:
First 4 tokens in labels: [50258, 50263, 50359, 50363]
✅ Labels contain CORRECT 4 prefix tokens

📊 TOKEN COUN

In [3]:
import torch
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

print(model.config)

WhisperConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "apply_spec_augment": false,
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "classifier_proj_size": 256,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "dtype": "float32",
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_feature_prob": 0.0,
  "mask_time_length": 10,
  "mask_time_min_masks": 2,
  "mask_time_prob": 0.05

In [7]:
import torch
from transformers import Speech2TextForConditionalGeneration

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-medium-mustc-multilingual-st",use_safetensors=True)

print(model.config)

Speech2TextConfig {
  "activation_dropout": 0.15,
  "activation_function": "relu",
  "architectures": [
    "Speech2TextForConditionalGeneration"
  ],
  "attention_dropout": 0.15,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "conv_channels": 1024,
  "conv_kernel_sizes": [
    5,
    5
  ],
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.15,
  "dtype": "float32",
  "early_stopping": true,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "input_channels": 1,
  "input_feat_per_channel": 80,
  "is_encoder_decoder": true,
  "max_length": 200,
  "max_source_positions": 6000,
  "max_target_positions": 1024,
  "model_type": "speech_to_text",
  "num_beams": 5,
  "num_conv_layers": 2,
  "num_hidden_layers": 12,
  "pad_token_id"